In [ ]:
import bpy
import numpy as np

def make_contreras_snail(label="snail", 
                         b = .1, d = 4, z = 1, a = 5, phi = 0, psi = 0, 
                         c_depth=0.1, c_n = 10, n_depth = 0, n = 0, 
                         h_0 = 1, eps = 0.5,
                         time = 20, n_points_time = 1000, 
                         n_points_aperture=100):

    # snail axes are [XYZ][Aperture Angle Theta][Time]

    _lambda: np.array = np.zeros((3, n_points_time, n_points_aperture))
    gamma: np.array = np.zeros((3, n_points_time, n_points_aperture))
    C: np.array = np.zeros((3, n_points_time, n_points_aperture))

    # vector of points in time reshaped for broadcasting
    t: np.array = np.linspace(0, time, n_points_time).reshape((n_points_time, 1))
    theta: np.array = np.linspace(0, 2*np.pi, n_points_aperture).reshape((1, n_points_aperture))

    # precalculating some repeated operations for efficiency
    sin_t = np.sin(t)
    cos_t = np.cos(t)
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    bsq = b**2
    dsq = d**2

    radial_ribbing = (1 + n_depth + n_depth*np.sin(n*theta))
    spiral_ribbing = (1 + c_depth + c_depth*np.sin(c_n*t))
    #spiral_ribbing = 1 + c_depth + (c_depth*np.sin(c_n*t)) + np.abs((c_depth*np.sin(c_n*t)))

    # shape = (3, n_points_time, 1) -> [xyz][ti)me][constant with respect to the angle theta along the aperture]
    gamma = np.exp(b*t)*np.array([
        d*sin_t, d*cos_t, 
        np.full(n_points_time, z).reshape(n_points_time, 1)
    ])

    # Defining the normal and binormal vectors along the frennet frame for all time points and angles about the aperture
    # 3 x n_points_time x 1 for both
    N: np.array = np.array([
        b*cos_t - sin_t,
        -b*sin_t - cos_t,
        np.zeros((n_points_time, 1))
    ])/np.sqrt(b**2 + 1)

    B: np.array = np.array([
        b*z*(b*sin_t + cos_t),
        b*z*(b*cos_t - sin_t),
        np.full((n_points_time, 1), d*(bsq + 1))
    ])/np.sqrt(
        (bsq + 1)*((bsq + 1)*dsq + bsq*(z**2))
    )

    # Define the rotation matrix for the aperture given that psi 
    # is the rotation angle about the B axis in the local frennet frame
    R: np.array = np.array([
        [np.cos(psi), -np.sin(psi), 0],
        [np.sin(psi), np.cos(psi), 0],
        [0, 0, 1]
    ])
    
    # Define the unrotated and unscaled generating curve
    # Shape = (1, 1, n_points_aperture) | constant for [xyz][time], [changes for the angle]
    GC: np.array = (a*sin_theta*np.sin(phi) - cos_theta*np.cos(phi)).reshape(1, 1, n_points_aperture)*B + (a*sin_theta*np.cos(phi) + cos_theta*np.sin(phi)).reshape(1, 1, n_points_aperture)*N
    GC_outer: np.array = radial_ribbing*GC 

    # einsum lets me broadcast the rotation matrix multiplication over the time points and the aperture angles
    rGC_outer = np.einsum('ij,jkl->ikl', R, GC_outer)
    rGC = np.einsum('ij,jkl->ikl', R, GC)
    timescale = (np.exp(b*t) - (1 / (t + 1)))
    #adjusting the wave height from 1 to (1+c_depth) so the percentage is always > 100%
    outer_timescale = spiral_ribbing*timescale
    # eps is a modified version of the shell width parameter described by Okabe 2017
    inner_timescale = timescale - ((timescale**eps)*h_0)

    # We tranpose the angle and time axes so the final shape is [XYZ][Aperture][Time], which is easier to index for mesh
    outer_mesh = np.transpose(gamma + (outer_timescale * rGC_outer), (0, 2, 1))
    inner_mesh = np.transpose(gamma + (inner_timescale * rGC), (0, 2, 1))

    # find the indices of the vertices and their faces that will be used for Blender mesh conversion
    indices = np.array(range(n_points_aperture*n_points_time))
    index_grid = indices.reshape(n_points_time, n_points_aperture).T
    v1 = index_grid[:, :-1]                       
    v2 = index_grid[:, 1:]                         
    v3 = np.roll(index_grid, -1, axis=0)[:, 1:]     
    v4 = np.roll(index_grid, -1, axis=0)[:, :-1]    
    # Stack the adjusted indices to form faces with vertical wrapping
    faces = np.stack((v1, v2, v3, v4), axis=-1).reshape(-1, 4)

    # Reshape the inner and outer meshes so their shapes are compatable with Blender
    return {"outer_mesh" : outer_mesh, 
            "inner_mesh" : inner_mesh,
            "outer_mesh_vertices": outer_mesh.reshape(3, n_points_aperture*n_points_time),
            "inner_mesh_vertices": inner_mesh.reshape(3, n_points_aperture*n_points_time),
            "vertex_indices": indices,
            "face_vertex_indices": faces
            }

In [104]:
snail = make_contreras_snail(z = 1.3, a = 1, d=1, phi=0, psi=0,
                             b=.15,
                             n_depth=0, n=0, 
                             c_n=0, c_depth=0,  
                             time=200, n_points_time=950, 
                             n_points_aperture=20, 
                             h_0 = 40, eps=.8)

In [109]:
snail["face_vertex_indices"]

array([[    0,    20,    21,     1],
       [   20,    40,    41,    21],
       [   40,    60,    61,    41],
       ...,
       [18939, 18959, 18940, 18920],
       [18959, 18979, 18960, 18940],
       [18979, 18999, 18980, 18960]])

In [ ]:




import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

%matplotlib qt

# Assuming snail dictionary has already been defined with the actual snail mesh data
outer_mesh = snail["outer_mesh"]
inner_mesh = snail["inner_mesh"]

# Extract X, Y, Z for the outer and inner meshes
X_outer, Y_outer, Z_outer = outer_mesh[0], outer_mesh[1], outer_mesh[2]
X_inner, Y_inner, Z_inner = inner_mesh[0], inner_mesh[1], inner_mesh[2]

# Creating a 3D plot of the snail mesh
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plotting the 3D surface of the outer snail mesh (outer layer) with slight transparency
ax.plot_surface(X_outer, Y_outer, Z_outer, rstride=1, cstride=1, cmap='viridis', edgecolor='none', alpha=0.6)

# Plotting the 3D surface of the inner snail mesh (inner layer, in red, fully opaque)
ax.plot_surface(X_inner, Y_inner, Z_inner, rstride=1, cstride=1, color='red', edgecolor='none', alpha=1.0)

# Setting labels and title
ax.set_xlabel('X Dimension')
ax.set_ylabel('Y Dimension')
ax.set_zlabel('Z Dimension')
ax.set_title('3D Surface Plot of the Contreras Snail')

# Calculating mean for each axis to center the limits
mean_x = np.mean(np.concatenate([X_outer.flatten(), X_inner.flatten()]))
mean_y = np.mean(np.concatenate([Y_outer.flatten(), Y_inner.flatten()]))
mean_z = np.mean(np.concatenate([Z_outer.flatten(), Z_inner.flatten()]))

# Calculating the range to ensure all axes have equal limits
all_data = np.concatenate([X_outer.flatten(), Y_outer.flatten(), Z_outer.flatten(),
                           X_inner.flatten(), Y_inner.flatten(), Z_inner.flatten()])
min_range = all_data.min() * 1.2
max_range = all_data.max() * 1.2
range_span = (max_range - min_range) / 2

# Setting the limits for all axes centered at their mean
ax.set_xlim(mean_x - range_span, mean_x + range_span)
ax.set_ylim(mean_y - range_span, mean_y + range_span)
ax.set_zlim(mean_z - range_span, mean_z + range_span)

# Setting the initial viewing angle to focus on the XY plane
ax.view_init(elev=90, azim=-90)

# Showing the plot
plt.show()
